In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import catboost as cb
from catboost import CatBoostClassifier
import time

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score, roc_auc_score, average_precision_score, precision_score, recall_score, classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, StratifiedKFold, cross_val_score

from cc5_preprocessor import Preprocessor

import joblib

np.random.seed(42)

In [2]:
df = pd.read_csv('data/Training_TriGuard.csv')
df = df.dropna(subset=['subrogation'])

In [3]:
pre = Preprocessor(smoothing_factor=5, mode = 'catboost')

In [4]:
X = df.drop(columns=["subrogation"]).copy()
y = df["subrogation"].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

In [5]:
pre.fit(X_train, y_train)

X_train_proc = pre.transform(X_train)
X_test_proc = pre.transform(X_test)

X_test_proc = X_test_proc.reindex(columns=X_train_proc.columns, fill_value=0)

Fitting Preprocessor in 'catboost' mode...
CatBoost mode: Skipping target encoding learning.
Learning statistical parameters for Z-scoring...
Fit complete.
Transforming data in 'catboost' mode...
Applying learned statistical transforms (Z-scores)...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.
Transforming data in 'catboost' mode...
Applying learned statistical transforms (Z-scores)...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.


In [6]:
pre.fit(X, y)
X_proc = pre.transform(X)

Fitting Preprocessor in 'catboost' mode...
CatBoost mode: Skipping target encoding learning.
Learning statistical parameters for Z-scoring...
Fit complete.
Transforming data in 'catboost' mode...
Applying learned statistical transforms (Z-scores)...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.


## CatBoost with Optuna Tuning

In [7]:
import optuna
from optuna.integration import CatBoostPruningCallback

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
CAT_FEATURES = pre.cat_for_encoding_
print(CAT_FEATURES)

['accident_site', 'accident_type', 'channel', 'vehicle_category', 'vehicle_color', 'living_status', 'claim_day_of_week', 'gender', 'in_network_bodyshop', 'season']


In [9]:


def objective(trial: optuna.trial.Trial) -> float:
    
    # 1. Define hyperparameters from Optuna
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.1),
        'random_strength': trial.suggest_float('random_strength', 1e-8, 1.0, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 10.0),
        
        # --- THIS IS THE FIX ---
        # Add the static parameters (including cat_features) here
        'eval_metric': 'F1',
        'task_type': 'CPU',
        'verbose': False,
        'random_state': 42,
        'cat_features': CAT_FEATURES  # <-- Pass cat_features at initialization
    }

    # 2. Set up the model
    # The model now knows about cat_features from the start
    model = CatBoostClassifier(**params)
    
    # 3. Configure CV Splitter
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42) # 3-fold for speed

    # 4. Run CV on the TRAINING data
    # We no longer need 'fit_params' because the model already has the info
    f1_scores = cross_val_score(
        model,
        X_proc,
        y,
        cv=skf,
        scoring='f1'
    )
    
    # 5. Return the mean F1 from the folds
    return np.mean(f1_scores)

In [10]:
print("\n2. Starting Optuna study...")

study = optuna.create_study(
    direction='maximize',
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10)
)

study.optimize(
    objective, 
    n_trials=100, # Number of trials to run
    show_progress_bar=True
)

print("\n" + "="*50)
print("Optuna study finished.")
print(f"Number of finished trials: {len(study.trials)}")

print("\nBest trial:")
best_trial = study.best_trial
    
print(f"  Value (Max F1 Score): {best_trial.value:.4f}") # <-- CHANGED comment
    
print("  Best Hyperparameters:")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

[I 2025-11-13 22:11:04,805] A new study created in memory with name: no-name-a3c13103-fb15-46d8-89ff-1c417739c5ef



2. Starting Optuna study...


Best trial: 0. Best value: 0.549104:   1%|          | 1/100 [00:34<57:11, 34.66s/it]

[I 2025-11-13 22:11:39,474] Trial 0 finished with value: 0.5491044579495765 and parameters: {'learning_rate': 0.08174445497072659, 'depth': 7, 'l2_leaf_reg': 0.6912461495176748, 'subsample': 0.8, 'random_strength': 0.027833425192154176, 'bagging_temperature': 0.38846852930913056, 'border_count': 176, 'scale_pos_weight': 4.823080042929025}. Best is trial 0 with value: 0.5491044579495765.


Best trial: 0. Best value: 0.549104:   2%|▏         | 2/100 [02:34<2:18:55, 85.06s/it]

...

[I 2025-11-13 23:12:36,901] Trial 97 finished with value: 0.5759571912949183 and parameters: {'learning_rate': 0.010792785073826429, 'depth': 3, 'l2_leaf_reg': 4.515891591654092, 'subsample': 0.6, 'random_strength': 0.05843989064174885, 'bagging_temperature': 0.2875937780278376, 'border_count': 106, 'scale_pos_weight': 1.5748536470838597}. Best is trial 82 with value: 0.5937182473977077.


Best trial: 82. Best value: 0.593718:  99%|█████████▉| 99/100 [1:01:44<00:12, 12.33s/it]

[I 2025-11-13 23:12:49,148] Trial 98 finished with value: 0.5908427909422572 and parameters: {'learning_rate': 0.010874779924323705, 'depth': 3, 'l2_leaf_reg': 9.683893934963763, 'subsample': 0.6, 'random_strength': 0.20682254620079887, 'bagging_temperature': 0.4052544869919372, 'border_count': 96, 'scale_pos_weight': 2.313413991257198}. Best is trial 82 with value: 0.5937182473977077.


Best trial: 82. Best value: 0.593718: 100%|██████████| 100/100 [1:01:56<00:00, 37.16s/it]

[I 2025-11-13 23:13:01,011] Trial 99 finished with value: 0.5845913474839703 and parameters: {'learning_rate': 0.012992137985040772, 'depth': 3, 'l2_leaf_reg': 3.575191698052905, 'subsample': 0.5, 'random_strength': 0.03129170062505502, 'bagging_temperature': 0.4094274372243147, 'border_count': 77, 'scale_pos_weight': 1.7797178386183823}. Best is trial 82 with value: 0.5937182473977077.

Optuna study finished.
Number of finished trials: 100

Best trial:
  Value (Max F1 Score): 0.5937
  Best Hyperparameters:
    learning_rate: 0.0146056237215288
    depth: 3
    l2_leaf_reg: 5.399340044266502
    subsample: 0.5
    random_strength: 0.3376358733478773
    bagging_temperature: 0.4663247387696527
    border_count: 135
    scale_pos_weight: 2.2636440427560536


In [11]:
best_params = study.best_trial.params
print(best_params)

final_params = best_params.copy()
final_params.update({
    'iterations': 2000, # Use more iterations for the final model
    'eval_metric': 'Logloss', # Use Logloss for training/stopping
    'task_type': 'CPU',
    'early_stopping_rounds': 100, # Keep early stopping
    'random_state': 123
})

best_model = CatBoostClassifier(**final_params)

best_model.fit(
    X_train_proc, y_train,
    eval_set=(X_test_proc, y_test),
    cat_features=CAT_FEATURES,
    verbose=False
)

print(f"\nFinal Model Score (from best Logloss iteration):")
y_preds_final = best_model.predict(X_test_proc)
final_f1 = f1_score(y_test, y_preds_final, pos_label=1)
print(f"  Manual F1:class=1 Score: {final_f1:.4f}")
        
print("\n  Full Classification Report:")
print(classification_report(y_test, y_preds_final, target_names=['Class 0.0', 'Class 1.0']))

{'learning_rate': 0.0146056237215288, 'depth': 3, 'l2_leaf_reg': 5.399340044266502, 'subsample': 0.5, 'random_strength': 0.3376358733478773, 'bagging_temperature': 0.4663247387696527, 'border_count': 135, 'scale_pos_weight': 2.2636440427560536}

Final Model Score (from best Logloss iteration):
  Manual F1:class=1 Score: 0.5782

  Full Classification Report:
              precision    recall  f1-score   support

   Class 0.0       0.89      0.80      0.85      4165
   Class 1.0       0.50      0.68      0.58      1235

    accuracy                           0.77      5400
   macro avg       0.70      0.74      0.71      5400
weighted avg       0.80      0.77      0.78      5400



In [12]:
print("Saving best_model...")

# Use the model's F1 score in the name
best_model.save_model("catboost_mod_f1_0.5937_cv.cbm")

print("Done.")

Saving best_model...
Done.


In [14]:
importances = best_model.get_feature_importance()
feature_names = best_model.feature_names_

feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
})

feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

print("--- Feature Importance Analysis ---")
print(f"Total features: {len(feature_importance_df)}")

print("\nTop 10 Most Important Features:")
print(feature_importance_df.head(10))

print("\nBottom 10 Least Important Features:")
print(feature_importance_df.tail(10))

n_features_to_remove = 100
n_features_to_keep = len(feature_importance_df) - n_features_to_remove
top_features = feature_importance_df.head(n_features_to_keep)['feature'].tolist()

print(f"\nKeeping top {len(top_features)} features and removing bottom {n_features_to_remove}.")

X_train_top_features = X_train_proc[top_features]
X_test_top_features = X_test_proc.reindex(columns=top_features, fill_value=0) 

original_cat_features = set(CAT_FEATURES)
top_features_set = set(top_features)
new_cat_features = list(original_cat_features.intersection(top_features_set))

print(f"Original categorical features: {len(CAT_FEATURES)}")
print(f"Categorical features kept: {len(new_cat_features)}")

best_params_from_optuna = study.best_trial.params

final_params_new = best_params_from_optuna.copy()
final_params_new.update({
    'iterations': 2000, 
    'eval_metric': 'Logloss',
    'task_type': 'CPU',
    'early_stopping_rounds': 50
})

print("\nRetraining model with top features...")
new_model = CatBoostClassifier(**final_params_new)

new_model.fit(
    X_train_top_features, y_train,
    eval_set=(X_test_top_features, y_test),
    cat_features=new_cat_features,
    verbose=False
)

y_preds_new = new_model.predict(X_test_top_features)
new_f1 = f1_score(y_test, y_preds_new, pos_label=1)

print("\n--- Model Performance Comparison ---")
# 'final_f1' comes from cell 30 in your notebook
print(f"Original F1 score (all features): {final_f1:.4f}")
print(f"New F1 score (top {len(top_features)} features): {new_f1:.4f}")

print("\nNew Model Classification Report (Top Features):")
print(classification_report(y_test, y_preds_new, target_names=['Class 0.0', 'Class 1.0']))

--- Feature Importance Analysis ---
Total features: 161

Top 10 Most Important Features:
                        feature  importance
152  recovery_feasibility_score    9.306899
59              liab_prct_cubed    6.711657
53          multicar_x_highrisk    6.621012
58            liab_prct_squared    6.408799
60               liab_prct_sqrt    6.031098
47              liab_x_multicar    5.400718
13                    liab_prct    5.391125
62                 liab_inverse    5.281119
61                liab_prct_log    5.067079
67     is_multi_vehicle_unclear    4.585798

Bottom 10 Least Important Features:
                     feature  importance
57   witness_police_multicar         0.0
111       first_time_claimer         0.0
76      evidence_very_strong         0.0
99             heavy_vehicle         0.0
50          witness_x_police         0.0
42             police_binary         0.0
133              high_income         0.0
110         moderate_claimer         0.0
135               low

In [15]:
# Output module, from model_citizens.ipynb
real_test = pd.read_csv("data/Testing_TriGuard.csv")

X_real_test_proc = pre.transform(real_test)
X_real_test_proc = X_real_test_proc.reindex(columns=X_train_proc.columns, fill_value=0)
real_pred_proba = best_model.predict_proba(X_real_test_proc)[:, 1]
real_pred_label = (real_pred_proba >= 0.5).astype(int)

prediction = pd.DataFrame({
    "claim_number": real_test["claim_number"],
    "subrogation": real_pred_label
})

print(prediction.head())

Transforming data in 'catboost' mode...
Applying learned statistical transforms (Z-scores)...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.
   claim_number  subrogation
0       3126034            0
1       7380142            1
2       4655051            0
3       6728725            1
4       9848460            1


In [16]:
prediction.to_csv("results/catboost_5937_cv_prediction.csv", index=False)